In [38]:
from bqplot import (
    Axis, Scatter, Lines, Figure, LinearScale
)
from ipywidgets import HBox
from collections import namedtuple
from numpy import add
sc_x = LinearScale(min=-5,max=5)
sc_y = sc_x

VectorControl = namedtuple('VectorControl',['vec','marks','addListener'])
def vector_control(v_coords,color='green'): 
    """
    An interactive vector.
    @return the marks making it up, and the method to update self
    addListener adds a function that is called when the vector is changed.
    """
    x_a,y_a = v_coords
    lin = Lines(x=[0,x_a], y=[0,y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color])
    head = Scatter(x=[x_a], y=[y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color],enable_move=True)
    update_funcs=[] #list of methods to be called when this vector is changed (like changeListeners)
    def vec():
        return (head.x[0],head.y[0])
    p = VectorControl(vec=vec,marks=[lin,head],addListener=lambda x: update_funcs.append(x))
    def update_self():
        lin.x=[0,head.x[0]]
        lin.y=[0,head.y[0]]
    head.update_on_move=True
    def update(change=None):
        update_self()
        for f in update_funcs:
            f()
    head.observe(update,names=['x','y'])
    return p

Vector = namedtuple('Vector',['getVec','marks'])
def dep_vector(color,update_func,*depended_on):
    """
    A dependent vector. dependent_on must implement function addListener
    The update_func receives all depended_on as parameters and returns an (x,y) tuple representing a vector.
    The update_func is also added as listener to all depended_on.
    """
    x_a,y_a = update_func(*depended_on)
    lin = Lines(x=[0,x_a], y=[0,y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color])
    head = Scatter(x=[x_a], y=[y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color],enable_move=False)
    def setVec(v_coords):
        head.x, head.y=[v_coords[0]], [v_coords[1]]
        lin.x,lin.y=[0,v_coords[0]], [0,v_coords[1]]
        
    def getVec():
        return (head.x[0],head.y[0])
    update_func_applied = lambda : setVec(update_func(*depended_on))
    for control in depended_on:
        control.addListener(update_func_applied)
    return Vector(getVec=getVec,marks=[lin,head])

def affine_dep_vec(color,update_func,*depended_on):
    """
    A dependent vector. dependent_on must implement function addListener
    The update_func receives all depended_on as parameters and returns an (x_o,y_o),(x_a,y_a) pair of tuples.
    The first is the origin, the second the head.
    The update_func is also added as listener to all depended_on.
    """
    (x_o,y_o),(x_a,y_a) = update_func(*depended_on)
    lin = Lines(x=[x_o,x_a], y=[y_o,y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color])
    head = Scatter(x=[x_a], y=[y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color],enable_move=False)
    #would be nice if we had lenses here
    def setVec(origin_v_coords):
        (x_o,y_o),(x_a,y_a) = origin_v_coords
        head.x, head.y=[x_a],[y_a]
        lin.x,lin.y=[x_o,x_a], [y_o,y_a]
        
    def getVec():
        return (head.x[0],head.y[0])
    update_func_applied = lambda : setVec(update_func(*depended_on))
    for control in depended_on:
        control.addListener(update_func_applied)
    return Vector(getVec=getVec,marks=[lin,head])



# Complex number multiplication with z(a+ib) = za +zib

In [39]:
z1 = vector_control((1,0),'green')
z2 = vector_control((0,1),'orange')

def c_mult(z1,z2):
    (x1,y1)=z1
    (x2,y2)=z2
    return (x1*x2-y1*y2,x1*y2+x2*y1)
cv_mult = lambda z1,z2: c_mult(z1.vec(),z2.vec())

stretched_z1=lambda z1,z2: c_mult(z1.vec(),(z2.vec()[0],0))
rotated_z1=lambda z1,z2: c_mult(z1.vec(),(0,z2.vec()[1]))

rot_translated_z1=lambda z1,z2: (stretched_z1(z1,z2),add(stretched_z1(z1,z2),rotated_z1(z1,z2)))

stretched=dep_vector('green',stretched_z1,z1,z2)
rot_tr=affine_dep_vec('orange', rot_translated_z1,z1,z2)
multiplied=dep_vector('white',cv_mult,z1,z2)

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, orientation='vertical')
fig1 = Figure(marks=z1.marks+z2.marks, axes=[ax_x, ax_y])
fig3 = Figure(marks= flatten(map((lambda x: x.marks), [stretched,rot_tr,multiplied])), axes=[ax_x, ax_y])
HBox([fig1,fig3])


# Components of Complex Number multiplication

In [19]:
#we really need to do this in a proper editor where we can do rename refactorings
z1 = vector_control((1,0),'green')
z2 = vector_control((0,1),'orange')

#ok how about this: 
#a dependent vector is initialized with vectors it depends on, and a function 
#that depends on those vectors that gets called with all of them as parameters
#whenever any one gets updated. This is something that would never work in Haskell, I think
#without some strange type family-Fu.
#alright, we can do this! Though I'm missing the functionality of my emacs elpy mode...

ac_func = lambda z1, z2: (z1.vec()[0]*z2.vec()[0],0)
bd_func = lambda z1, z2: (-1*z1.vec()[1]*z2.vec()[1],0)
ad_func = lambda z1, z2: (0,z1.vec()[0]*z2.vec()[1])
cb_func = lambda z1, z2: ad_func(z2,z1)

prod_func = lambda z1, z2: add.reduce((ac_func(z1,z2),bd_func(z1,z2),ad_func(z1,z2),cb_func(z1,z2)))

ac = dep_vector('red',ac_func,z1,z2)
bd = dep_vector('blue',bd_func,z1,z2)
ad = dep_vector('red',ad_func,z1,z2)
cb = dep_vector('blue',cb_func,z1,z2)

prod = dep_vector('yellow',prod_func,z1,z2)

#from: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flatten = lambda l: [item for sublist in l for item in sublist]

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, orientation='vertical')
fig1 = Figure(marks=z1.marks+z2.marks, axes=[ax_x, ax_y])
fig3 = Figure(marks= flatten(map((lambda x: x.marks), [ac,bd,ad,cb,prod])), axes=[ax_x, ax_y])
HBox([fig1,fig3])